<center>
    <img src="img/scikit-learn-logo.png" width="40%" />
    <br />
    <h1>An introduction to Machine Learning with Scikit-Learn</h1>
    <br /><br />
    Gilles Louppe (<a href="https://twitter.com/glouppe">@glouppe</a>), April 23, 2015
    <br /><br />
    <a href="https://github.com/glouppe/tutorial-sklearn-lhcb">https://github.com/glouppe/tutorial-sklearn-lhcb</a>
</center>

# About myself 

* PhD in machine learning, expertise in tree-based methods
* Core developer of Scikit-Learn
* Research Fellow working in GS-SIS, on graph and text mining problems
* Open to collaborations

# Outline

* What is machine learning?
* Scikit-Learn
* Supervised learning
* Model evaluation and selection
* Unsupervised learning

# What is machine learning?

# Scikit-Learn

## Python stack for data analysis

## Scikit-Learn

## A simple and unified API

- instantiation with hyperparameters
- learning with fit (learning the parameters)
- predictions

# Supervised learning algorithms

## Classification with linear models

## Regression with tree-based methods

# Model evaluation and selection

- Selection versus evaluation
- Parameter search
- Cross-validation
- Under/over-fitting
- Metrics

## Selection and evaluation

## Parameter search

## Under/over-fitting

## Cross-validation

# Unsupervised learning

## Clustering

## Density estimation

## Decomposition

# Going further

- Add references to tutorials, packages, etc